In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from string import ascii_lowercase
from itertools import combinations

import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE

In [4]:
! pip install impyute

In [5]:
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [6]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

In [7]:
train_quality_df = train_quality.drop(['time','fwver'],axis=1)
train_quality_df.fillna(0,inplace=True)

def string2num(x):
  x = re.sub(r"[^0-9]+",'',str(x))
  if x == '':
    return 0
  else:
    return int(x)

train_quality_df['quality_5'] = train_quality_df['quality_5'].apply(lambda x : string2num(x))
train_quality_df['quality_7'] = train_quality_df['quality_7'].apply(lambda x : string2num(x))
train_quality_df['quality_8'] = train_quality_df['quality_8'].apply(lambda x : string2num(x))
train_quality_df['quality_9'] = train_quality_df['quality_9'].apply(lambda x : string2num(x))
train_quality_df['quality_10'] = train_quality_df['quality_10'].apply(lambda x : string2num(x))

train_quality_df['quality_5'] = train_quality_df['quality_5'].astype('int')
train_quality_df['quality_7'] = train_quality_df['quality_7'].astype('int')
train_quality_df['quality_8'] = train_quality_df['quality_8'].astype('int')
train_quality_df['quality_9'] = train_quality_df['quality_9'].astype('int')
train_quality_df['quality_10'] = train_quality_df['quality_10'].astype('int')

mean_train_quality = train_quality_df.groupby('user_id').agg('median')
mean_train_quality = mean_train_quality.reset_index()

mean_train_quality.drop(['quality_3','quality_4'],axis=1,inplace = True)

id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-10000,err-1]+=1

df_error = pd.DataFrame(error)
df_error['user_id'] = train_err['user_id']

problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1 

merged_train_err_quality = pd.merge(df_error,mean_train_quality,on='user_id',how='left')
del merged_train_err_quality['user_id']

multiple_imputation_df = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_train_err_quality))

multiple_imputation_df.drop([44,20,43,8,1,20,19,7],axis=1,inplace=True)

train_x = multiple_imputation_df.values
train_y = problem
print(train_x.shape)
print(train_y.shape)

smote = SMOTE(random_state=0)
x_train_over, y_train_over = smote.fit_sample(train_x,train_y)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:',x_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포:\n',pd.Series(y_train_over).value_counts())

100%|██████████| 16554663/16554663 [00:47<00:00, 350470.00it/s]


(15000, 46)
(15000,)
SMOTE 적용 전 학습용 피처/레이블 데이터 세트: (20000, 46) (20000,)
SMOTE 적용 후 레이블 값 분포:
 1.0    10000
0.0    10000
dtype: int64


In [8]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

test_quality_df = test_quality.drop(['time','fwver'],axis=1)

test_quality_df['quality_1'] = test_quality_df['quality_1'].apply(lambda x : string2num(x))
test_quality_df['quality_5'] = test_quality_df['quality_5'].apply(lambda x : string2num(x))
test_quality_df['quality_7'] = test_quality_df['quality_7'].apply(lambda x : string2num(x))
test_quality_df['quality_8'] = test_quality_df['quality_8'].apply(lambda x : string2num(x))
test_quality_df['quality_9'] = test_quality_df['quality_9'].apply(lambda x : string2num(x))
test_quality_df['quality_10'] = test_quality_df['quality_10'].apply(lambda x : string2num(x))

test_quality_df['quality_1'] = test_quality_df['quality_1'].astype('int')
test_quality_df['quality_5'] = test_quality_df['quality_5'].astype('int')
test_quality_df['quality_7'] = test_quality_df['quality_7'].astype('int')
test_quality_df['quality_8'] = test_quality_df['quality_8'].astype('int')
test_quality_df['quality_9'] = test_quality_df['quality_9'].astype('int')
test_quality_df['quality_10'] = test_quality_df['quality_10'].astype('int')

test_quality_df.fillna(0,inplace=True)

mean_test_quality = test_quality_df.groupby('user_id').agg('median')
mean_test_quality = mean_test_quality.reset_index()

mean_test_quality.drop(['quality_3','quality_4'],axis=1,inplace=True)

id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)

df_error_test = pd.DataFrame(test_x)
df_error_test['user_id'] = test_err['user_id']

merged_test_err_quality = pd.merge(df_error_test,mean_test_quality,on='user_id',how='left')
del merged_test_err_quality['user_id']

multiple_imputation_df_test = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_test_err_quality))

multiple_imputation_df_test.drop([44,20,43,8,1,20,19,7],axis=1,inplace=True)
test_x = multiple_imputation_df_test

100%|██████████| 16532648/16532648 [00:47<00:00, 345989.98it/s]


In [11]:
k_fold = KFold(n_splits = 5, shuffle = True, random_state = 0)

clf1 = RandomForestClassifier(n_estimators=500)
clf2 = LGBMClassifier()
clf3 = GradientBoostingClassifier()
soft_vote  = VotingClassifier([('r1',clf1), ('r2', clf2), ('r3',clf3)], voting='soft')
soft_vote.fit(x_train_over, y_train_over)

model = soft_vote
pred_y = model.predict_proba(test_x)
pred_y = pred_y[:,1]

submission = pd.DataFrame({
    'user_id': sample_submission['user_id'],
    "problem" : pred_y
})
submission.to_csv('/content/drive/MyDrive/데이콘/submission/model1.csv', index=False)

In [12]:
submission

,user_id,problem
0,30000,0.911619
1,30001,0.331993
2,30002,0.311964
3,30003,0.786786
4,30004,0.664406
...,...,...
14994,44994,0.494376
14995,44995,0.601515
14996,44996,0.649508
14997,44997,0.874532


In [45]:
test_x = pd.DataFrame(test_x.values)
test_x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,16.0,16.0,0.0,3.0,67.0,37.0,9.0,3.0,0.0,983.0,1308.0,5.0,2.0,32.0,0.0,0.0,0.0,0.0,83.0,2.0,16.0,0.0,1.0,3.0,3.0,0.0,2.0,92.0,0.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
1,0.0,3.0,98.0,3.0,4.0,4.0,0.0,12.0,16.0,4.0,6.0,53.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
2,0.0,0.0,0.0,79.0,2.0,1.0,0.0,16.0,16.0,0.0,1.0,173.0,133.0,0.0,0.0,0.0,60.0,40.0,2.0,0.0,40.0,0.0,0.0,0.0,0.0,230.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,14.0,113.0,1.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,107.0,2.0,2.0,0.0,4.0,5.0,1.0,7.0,8.0,12.0,0.0,1.0,0.0,3.0,2.0,0.0,0.0,5.0,2.0,2.0,0.0,0.0,134.0,0.0,4.0,16.0,3.0,1.0,1.0,0.0,0.0,40.0,8.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,2.0,6.0,1.0,1.0,0.0,10.0,12.0,1.0,18.0,80.0,67.0,6.0,0.0,0.0,10.0,8.0,0.0,0.0,14.0,1.0,1.0,0.0,0.0,332.0,152.0,11.0,6.0,1.0,1.0,1.0,1.0,0.0,48.0,90.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,2.0,12.0,5.0,5.0,0.0,16.0,19.0,3.0,9.0,123.0,93.0,12.0,0.0,0.0,197.0,204.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0,179.0,2.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,87.0,0.0,0.0,0.0,0.0,22.0,1.0,0.0,0.0,0.0,0.0
14995,0.0,0.0,0.0,0.0,2.0,2.0,0.0,14.0,15.0,1.0,4.0,149.0,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,183.0,0.0,14.0,1.0,0.0,1.0,1.0,0.0,0.0,12.0,17.0,0.0,0.0,0.0,0.0,22.0,1.0,0.0,0.0,0.0,0.0
14996,0.0,0.0,0.0,0.0,1.0,1.0,0.0,12.0,13.0,1.0,3.0,129.0,90.0,2.0,0.0,0.0,138.0,165.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,163.0,0.0,12.0,1118.0,0.0,1.0,1.0,1.0,0.0,329.0,2.0,3.0,0.0,0.0,0.0,22.0,1.0,0.0,0.0,0.0,0.0
14997,0.0,0.0,1.0,4.0,2.0,2.0,0.0,15.0,16.0,0.0,1.0,73.0,44.0,0.0,1.0,0.0,11127.0,13161.0,0.0,3.0,44.0,0.0,0.0,0.0,0.0,78.0,0.0,14.0,0.0,0.0,1.0,1.0,3.0,0.0,30.0,49.0,0.0,0.0,0.0,0.0,22.0,1.0,0.0,0.0,0.0,0.0


In [20]:
x_train_over = pd.DataFrame(x_train_over)
y_train_over = pd.DataFrame(y_train_over)

In [29]:
def lgbm_rfe_4040(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=4040)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)        
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    print(feats)
    return archive

In [30]:
lgbm_archive_4040 = lgbm_rfe_4040(x_train_over, y_train_over)

46 0.9123508551409256
41 0.9101642825462675
[12, 11, 20, 8, 3, 25, 13, 2, 28, 7, 34, 36, 10, 14, 18, 9, 17, 16, 35, 29, 38, 43, 27, 6, 40, 30, 21, 4, 24, 5, 32, 1, 26, 37, 41, 0, 39, 15, 44, 19, 22]


In [46]:
model = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_1 = x_train_over[lgbm_archive_4040.iloc[1,2]]

model.fit(x_train_1, y_train_over)

pred_y1 = model.predict_proba(test_x[lgbm_archive_4040.iloc[1,2]])
pred_y1 = pred_y1[:,1]

In [47]:
pred_y1

array([0.99929852, 0.10334381, 0.06450517, ..., 0.92723369, 0.97182134,
       0.86765442])

In [48]:
def lgbm_rfe_1234(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=1234)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [49]:
lgbm_archive_1234 = lgbm_rfe_1234(x_train_over, y_train_over)

46 0.8991969911087557
41 0.9010378665110123


In [51]:
model2 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_2 = x_train_over[lgbm_archive_1234.iloc[1,2]]

model2.fit(x_train_2, y_train_over)

pred_y2 = model2.predict_proba(test_x[lgbm_archive_1234.iloc[1,2]])
pred_y2 = pred_y2[:,1]

In [52]:
pred_y2

array([0.99860612, 0.36058379, 0.05144646, ..., 0.89191368, 0.9925789 ,
       0.86556052])

In [53]:
def lgbm_rfe_99087(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=99087)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [54]:
lgbm_archive_99087 = lgbm_rfe_99087(x_train_over, y_train_over)

46 0.9076474632812922
41 0.9064473617046946


In [56]:
model3 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_3 = x_train_over[lgbm_archive_99087.iloc[1,2]]

model3.fit(x_train_3, y_train_over)

pred_y3 = model3.predict_proba(test_x[lgbm_archive_99087.iloc[1,2]])
pred_y3 = pred_y3[:,1]

In [57]:
pred_y3

array([0.9953274 , 0.29664426, 0.07245987, ..., 0.8217962 , 0.91206668,
       0.34207381])

In [58]:
def lgbm_rfe_42(x_data, y_data, ratio=0.9, min_feats=40):
    feats = x_data.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(x_data[feats], y_data, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model': model, 'n_feats': n_feats, 'feats': feats, 'score': score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats * ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [61]:
lgbm_archive_42 = lgbm_rfe_42(x_train_over, y_train_over)

46 0.9068173843573613
41 0.9056700025374482


In [62]:
model4 = LGBMClassifier(objective="binary", num_iterations= 10**3)

x_train_4 = x_train_over[lgbm_archive_42.iloc[1,2]]

model4.fit(x_train_4, y_train_over)

pred_y4 = model4.predict_proba(test_x[lgbm_archive_42.iloc[1,2]])
pred_y4 = pred_y4[:,1]

In [63]:
pred_y4

array([0.99832333, 0.11529268, 0.07342833, ..., 0.92340373, 0.92307438,
       0.79636468])

In [64]:
pred_all = (pred_y + pred_y2 + pred_y3 + pred_y4) * (1/4)

submission = pd.DataFrame({
    "user_id" : sample_submission['user_id'],
    "problem" : pred_all
})
submission.to_csv('/content/drive/MyDrive/데이콘/submission/model2.csv', index=False)

In [65]:
pred_all

array([0.97596904, 0.27612853, 0.12732455, ..., 0.82165548, 0.92556296,
       0.64479875])

In [68]:
model1 = pd.read_csv('/content/drive/MyDrive/데이콘/submission/model1.csv', index_col = 'user_id')
model2 = pd.read_csv('/content/drive/MyDrive/데이콘/submission/model2.csv', index_col='user_id')

pred_y = (model1)*(0.7) + (model2)*(0.3)

submission = pd.DataFrame({
    'user_id': sample_submission['user_id'],
    'problem': pred_y['problem']
    })

submission.to_csv('/content/drive/MyDrive/데이콘/submission/combined_model1_model2.csv', index=False)

In [69]:
pred_y

,problem
user_id,
30000,0.930924
30001,0.315234
30002,0.256572
30003,0.811423
30004,0.729075
...,...
44994,0.506392
44995,0.615692
44996,0.701152


In [71]:
model3 = pd.read_csv('/content/drive/MyDrive/데이콘/submission/lgb_dart8_gbdt1_xgb1_stacking_smote_submission.csv', index_col='user_id')
del model3['Unnamed: 0']
model3

,problem
user_id,
30000,0.920411
30001,0.280516
30002,0.293975
30003,0.821893
30004,0.735171
...,...
44994,0.323453
44995,0.390028
44996,0.528600


In [72]:
fnl_pred_y = (pred_y * 0.2) + (model3 *0.8)
fnl_pred_y

,problem
user_id,
30000,0.922514
30001,0.287459
30002,0.286494
30003,0.819799
30004,0.733952
...,...
44994,0.360041
44995,0.435161
44996,0.563110


In [84]:
fnl_pred_y.to_csv('/content/drive/MyDrive/데이콘/submission/fnl_pred_y.csv', index=False)

In [80]:
df = pd.merge(model3,fnl_pred_y, on = 'user_id')
df

,problem_x,problem_y
user_id,,
30000,0.920411,0.922514
30001,0.280516,0.287459
30002,0.293975,0.286494
30003,0.821893,0.819799
30004,0.735171,0.733952
...,...,...
44994,0.323453,0.360041
44995,0.390028,0.435161
44996,0.528600,0.563110


In [82]:
df['error'] = (df['problem_x']- df['problem_y'])**2
df['error'].sum()

1.3504368753583444

In [83]:
df

,problem_x,problem_y,error
user_id,,,
30000,0.920411,0.922514,0.000004
30001,0.280516,0.287459,0.000048
30002,0.293975,0.286494,0.000056
30003,0.821893,0.819799,0.000004
30004,0.735171,0.733952,0.000001
...,...,...,...
44994,0.323453,0.360041,0.001339
44995,0.390028,0.435161,0.002037
44996,0.528600,0.563110,0.001191
